# Domain Scrape

In [1]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json

In [2]:
headers = {"User-Agent":
           "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# home url of domian.com australia
home_url = "https://www.domain.com.au"

In [3]:
#postcode of properties
postcode_list = [i for i in range(3000,3100)]

In [4]:
# return unique extracted link for given postcode
def link_by_postcode(postcode):
    
    # list to store all the urls of properties
    list_of_links = []
    
    # store unique link
    abc_links = [] 
    
    # number of pages of search result are 50
    page_numbers = list(range(50))[1:50]
    
    # for loop for all 50 search(melbourne region) pages
    for page in page_numbers:

        # extracting html document of search page
        url = home_url + f"/rent/melbourne-vic-{postcode}/?excludedeposittaken=1&page={page}"

        # parsing html document to 'lxml' format
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        #for each page
        # finding all the links available in 'ul' tag whos 'data-testid' is 'results'
        all_links = bs_object.find(
            "ul", {"data-testid": "results"}).findAll("a", href=re.compile("https://www.domain.com.au/*"))

        # inner loop to find links inside each property page because few properties are project so they have more properties inside their project page
        for link1 in all_links:
            # checking if it is a project and then performing similar above
            if 'project' in link1.attrs['href']:
                inner1_url = link1.attrs['href']
                inner1_bsobj = BeautifulSoup(requests.get(inner1_url, headers=headers).text, "html.parser")
                for link2 in inner1_bsobj.find("div", {"name": "listing-details__other-listings"}).findAll("a", href=re.compile("https://www.domain.com.au/*")):
                    if 'href' in link2.attrs:
                        list_of_links.append(link2.attrs['href'])
            else:
                list_of_links.append(link1.attrs['href'])
   
    # remove deplicated links
    for i in list_of_links: 
        if i not in abc_links: 
            abc_links.append(i) 
    return abc_links


In [5]:
links_test =link_by_postcode(3000)

In [6]:
# defining required regural expression for data extraction     
pattern = re.compile(r'>(.+)<!.*>(.+?)</span>.*')
pattern1 = re.compile(r'>(.+)<.')
pattern2 = re.compile(r'destination=(.+)" rel=.')
#school
pattern_school = re.compile(r'>(.+)<')
pattern_distance = re.compile(r'">(.+)<!')
#street information
pattern_address1 = re.compile(r'>(.+)<!.*>')
pattern_address2 = re.compile(r'>(.+)<.')

In [7]:
basic_feature_list = []


# loop to iterate through each url
for link in links_test[:10]:
    
    # opening urls
    bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")
    
    # extracting address/name of property
    property_name = bs_object.find("h1", {"class": "css-164r41r"})
    
    # extracting baths, rooms, parking etc
    all_basic_features = bs_object.find("div", {"class": "css-ghc6s4"}).findAll("span", {"data-testid": "property-features-text-container"})
    
    # extracting property price
    property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})
    
    # extracting latitudes and longitudes
    lat_long = bs_object.find("a", {"target": "_blank", 'rel': "noopener noreferer"})
    
    #extract schoolname, school distance
    # school= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("h5",{"class": "css-5w5cop"})
    school_data = json.loads(bs_object.find("script", {"id": "__NEXT_DATA__"}).text)
    school = [s['name'] for s in school_data['props']['pageProps']['componentProps']['schoolCatchment']['schools']]
    school_distance= [s['distance'] for s in school_data['props']['pageProps']['componentProps']['schoolCatchment']['schools']]
  
    # address info
    address1 = bs_object.find("span", {"data-testid": "address-line1"})
    address2 = bs_object.find("span", {"data-testid": "address-line2"})
    
    #recent rented
    recent_rented_number = bs_object.find("div", {"class": "css-12uo2x5"})
    
    # dictionary to store temporary data
    basic_feature_dict = {}
    
    # few properties does not contain all the 4 features such as rooms, baths, parkings, area. So need to check
    # how many features they contain
    if len(all_basic_features) == 4:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[3]))[0][1]] = pattern.findall(str(all_basic_features[3]))[0][0]
        
    elif len(all_basic_features) == 3:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        
    elif len(all_basic_features) == 2:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        
    elif len(all_basic_features) == 1:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]

        
        
    # putting 'none' if school is missing    
    if school is None:
        basic_feature_dict['school'] = None
        
    else:
        basic_feature_dict['school'] = school
    
    # putting 'none' if school distance is missing   
    if school_distance is None:
        basic_feature_dict['school_distance'] = None
        
    else:
        basic_feature_dict['school_distance'] = school_distance
     
    
    
    # putting 'none' if address(street) distance is missing   
    if address1 is None:
        basic_feature_dict['address1'] = None
        
    else:
        basic_feature_dict['address1'] = pattern_address1.findall(str(address1))
        
    # putting 'none' if address2 is missing   
    if address2 is None:
        basic_feature_dict['address2'] = None
        
    else:
        basic_feature_dict['address2'] = sum([pattern_address2.findall(str(i)) for i in address2],[])
    
        # putting 'none' if recent_rented number is missing   
    if recent_rented_number is None:
        basic_feature_dict['recent_rented_number'] = None
        
    else:
        basic_feature_dict['recent_rented_number'] = pattern1.findall(str(recent_rented_number))[0]


    # putting 'none' if price is missing    
    if property_price is None:
        basic_feature_dict['price'] = None
        
    else:
        basic_feature_dict['price'] = pattern1.findall(str(property_price))[0]
        
    # putting 'none' if property name/address is missing       
    if property_name is None:
        basic_feature_dict['name'] = None
        
    else:
        basic_feature_dict['name'] = pattern1.findall(str(property_name))[0]
        
    # putting 'none' if latitude and logitude are missing        
    if lat_long is None:
        basic_feature_dict['lat'] = None
        basic_feature_dict['long'] = None
        
    else:
        basic_feature_dict['lat'] = pattern2.findall(str(lat_long))[0].split(',')[0]
        basic_feature_dict['long'] = pattern2.findall(str(lat_long))[0].split(',')[1]
        
    # appending all the data into a list
    basic_feature_list.append(basic_feature_dict)

In [8]:
# creating empty lists
beds_list = []
baths_list = []
parking_list = []
area_list = []
name_list = []
lat_list = []
long_list = []
price_list = []
school_list = []
school_distance_list =[]
address1_list = []
address2_list = []
recent_rented_number_list = []
recent_rented_price_list = []

# interating through list created above with data
for row in basic_feature_list:
    
    # checking if the row cointains 'Beds', 'Bed' or nothing
    if 'Beds' in row:
        beds_list.append(row['Beds'])
    elif 'bed' in row:
        beds_list.append(row['Bed'])
    else:
        beds_list.append(None)
        
    # checking if the row cointains 'Baths', 'Bath' or nothing    
    if 'Baths' in row:
        baths_list.append(row['Baths'])
    elif 'Bath ' in row:
        baths_list.append(row['Bath'])
    else:
        baths_list.append(None)
        
    # checking if the row cointains 'Parking', '-' or nothing     
    if 'Parking' in row and row['Parking'] != '−':
        parking_list.append(row['Parking'])
    else:
        parking_list.append(None)
        
    # checking if the row cointains ' ', or nothing. Because empty space (i.e. ' ') reprsents area  
    if ' ' in row:
        area_list.append(row[' '])
    else:
        area_list.append(None)
    # checking if the row cointains 'name' that is address of property         
    if 'name' in row:
        name_list.append(row['name'])
    else:
        name_list.append(None)
    
    # checking if the row cointains 'price'         
    if 'price' in row:
        price_list.append(row['price'])
    else:
        price_list.append(None)        
    
    # checking if the row cointains 'lat' that is lattitude of property         
    if 'lat' in row:
        lat_list.append(row['lat'])
    else:
        lat_list.append(None)  
        
    # checking if the row cointains 'long' that is lattitude of property             
    if 'long' in row:
        long_list.append(row['long'])
    else:
        long_list.append(None)
        
    # checking if the row cointains 'school' that is lattitude of property             
    if 'school' in row:
        school_list.append(row['school'])
    else:
        school_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'school_distance' in row:
        school_distance_list.append(row['school_distance'])
    else:
        school_distance_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'address1' in row:
        address1_list.append(row['address1'])
    else:
        address1_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'address2' in row:
        address2_list.append(row['address2'])
    else:
        address2_list.append(None)
        
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'recent_rented_number' in row:
        recent_rented_number_list.append(row['recent_rented_number'])
    else:
        recent_rented_number_list.append(None)
    
    # checking if the row cointains 'scholl_distance' that is lattitude of property             
    if 'recent_rented_price' in row:
        recent_rented_price_list.append(row['recent_rented_price'])
    else:
        recent_rented_price_list.append(None)

In [9]:
#create dataframe 
house_dict = {}
house_dict['Beds'] = beds_list
house_dict['Baths'] = baths_list
house_dict['Parking'] = parking_list
house_dict['Area'] = area_list
house_dict['Address'] = name_list
house_dict['Latitude'] = lat_list
house_dict['Longitude'] = long_list
house_dict['Rent'] = price_list
house_dict['School'] = school_list
house_dict['School_distance'] = school_distance_list
house_dict['Location_1'] = address1_list
house_dict['Location_2'] = address2_list
house_dict['recent_rented_number'] = recent_rented_number_list
house_dict['recent_rented_price'] = recent_rented_price_list
house_df = pd.DataFrame(house_dict)

In [10]:
def remove_chars(s):
    return re.sub('[^0-9.]+', '', s) 

def remove_squaremeter(s):
    if s != None:
        return remove_chars(s)
    else:
        return None

In [11]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)


house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
house_df['Type'] = [type_flag(i) for i in price_list]

In [12]:
#extract post code
house_df['Postcode'] = house_df['Address'].str[-4:]

#clean area
house_df['Area'] = [remove_squaremeter(i) for i in area_list]

In [13]:
house_df

Beds Baths Parking  Area                                      Address  \
0     2  None    None  None  605/455 Elizabeth Street Melbourne VIC 3000   
1  None  None       1  None     309/9 Commercial Road Melbourne VIC 3004   
2     2     2       1  None     314/499 St Kilda Road Melbourne VIC 3004   
3  None  None       1  None  Unit 1713/200 Spencer St Melbourne VIC 3000   
4  None  None    None  None         701/101 Therry St Melbourne VIC 3000   
5     2     2    None  None         816/101 Therry St Melbourne VIC 3000   
6     2     2       2  None     303/250 St Kilda Road Melbourne VIC 3004   
7  None  None       1  None        805/81 Queens Road Melbourne VIC 3004   
8     2     2       1  None   3902/220 Spencer Street Melbourne VIC 3000   
9  None  None    None  None   3405/288 Spencer Street Melbourne VIC 3000   

      Latitude    Longitude                                       Rent  \
0  -37.8084293  144.9602064                              $500 per week   
1  -37.8456125  144.9793561                              $500 per week   
2  -37.8431328  144.9793408                $900 per week (unfurnished)   
3   -37.816228  144.9532465                              $440 per week   
4  -37.8074149  144.9591227        NEW corner apartment Keep your bond   
5  -37.8074149  144.9591227  New apartment No bond needed Pets welcome   
6  -37.8259165  144.9705248                             $1395 Per Week   
7   -37.853804  144.9798785                                $380 weekly   
8  -37.8157157  144.9529559                              $650 per week   
9   -37.813775  144.9520948                                       $440   

                                              School  \
0  [Eltham College - Lonsdale Street Campus, Rive...   
1  [SEDA College (Victoria) - Tennis - Albert Par...   
2  [SEDA College (Victoria) - Tennis - Albert Par...   
3  [Eltham College - King Street Campus, Ozford C...   
4  [River Nile School, Simonds Catholic College, ...   
5  [River Nile School, Simonds Catholic College, ...   
6  [Hester Hornbrook Academy, Melbourne Grammar S...   
7  [Wesley College - St Kilda Road Junior Campus,...   
8  [Eltham College - King Street Campus, Ozford C...   
9  [Ozford College - Ozford College Campus, Ozfor...   

                                     School_distance          Location_1  \
0  [463.3611599786358, 588.5331765906494, 695.270...  [455 Elizabeth St]   
1  [196.76699981293854, 433.2955593802291, 726.22...   [9 Commercial Rd]   
2  [147.59638689110963, 586.8570067562501, 678.78...   [499 St Kilda Rd]   
3  [241.54098639384102, 397.47737499083695, 398.5...    [200 Spencer St]   
4  [444.35865271135503, 572.3778370522139, 577.38...         [Therry St]   
5  [444.35865271135503, 572.3778370522139, 577.38...         [Therry St]   
6  [745.7774466794033, 866.7220566327758, 961.355...                None   
7  [246.02949240830281, 586.9835254409847, 1034.4...      [81 Queens Rd]   
8  [291.8975714410087, 352.5634804374277, 353.655...    [220 Spencer St]   
9  [240.2562788817361, 241.10246190089674, 307.86...    [288 Spencer St]   

               Location_2 recent_rented_number recent_rented_price  \
0  [Melbourne, VIC, 3000]                   81                None   
1  [Melbourne, VIC, 3004]                   71                None   
2  [Melbourne, VIC, 3004]                  276                None   
3  [Melbourne, VIC, 3000]                  506                None   
4  [Melbourne, VIC, 3000]                 1380                None   
5  [Melbourne, VIC, 3000]                 1380                None   
6                    None                 None                None   
7  [Melbourne, VIC, 3004]                   78                None   
8  [Melbourne, VIC, 3000]                 1151                None   
9  [Melbourne, VIC, 3000]                  315                None   

  Converted_Rent  Type Postcode  
0            500   1.0     3000  
1            500   1.0     3004  
2            900   1.0  

In [14]:
house_df['School'][0]

['Eltham College - Lonsdale Street Campus',
 'River Nile School',
 'Ozford College',
 'Ozford College - Ozford College Campus',
 'Simonds Catholic College',
 'Haileybury Girls College - City Campus',
 "St Joseph's Flexible Learning Centre",
 "Stott's Colleges",
 'Eltham College - King Street Campus',
 'Holmes Grammar School',
 'Carlton Gardens Primary School',
 'University High School']

In [15]:
house_df['School_distance'][0]

[463.3611599786358,
 588.5331765906494,
 695.2706405483688,
 696.2817703768635,
 700.2580984189686,
 700.4924347307267,
 724.9315920000029,
 976.7186990778307,
 1046.7892028267775,
 1082.5430994556723,
 1083.2267912562145,
 1329.1968129939876]

In [16]:
bs_object = BeautifulSoup(
    requests.get("https://www.domain.com.au/1-watson-street-armadale-vic-3143-16061315",
                 headers=headers).text, "html.parser")

In [17]:
school_data = json.loads(bs_object.find("script", {"id": "__NEXT_DATA__"}).text)
school = [s['name'] for s in school_data['props']['pageProps']['componentProps']['schoolCatchment']['schools']]
school

['Loreto Mandeville Hall',
 'Armadale Primary School',
 'Our Lady of Lourdes School',
 'The King David School - The Rebecca Magid Centre',
 "Lauriston Girls' School",
 'The King David School',
 "Lauriston Girls' School - Blairholme Campus",
 'The King David School - Junior School',
 'Geelong Grammar School - Toorak campus',
 "St Catherine's School",
 'De La Salle College - Kinnoull Campus',
 'Auburn High School',
 'Port Phillip Specialist School',
 'Southern Autistic School']

In [18]:
schools = pd.DataFrame(school)

In [19]:
schools

0
0                             Loreto Mandeville Hall
1                            Armadale Primary School
2                         Our Lady of Lourdes School
3   The King David School - The Rebecca Magid Centre
4                            Lauriston Girls' School
5                              The King David School
6        Lauriston Girls' School - Blairholme Campus
7              The King David School - Junior School
8             Geelong Grammar School - Toorak campus
9                              St Catherine's School
10             De La Salle College - Kinnoull Campus
11                                Auburn High School
12                    Port Phillip Specialist School
13                          Southern Autistic School

In [20]:
school= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("h5",{"class": "css-5w5cop"})

In [21]:
school

[<h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Armadale Primary School</h5>,
 <h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Auburn High School</h5>,
 <h5 class="css-5w5cop" data-testid="fe-co-school-catchment-school-title" itemprop="name">Port Phillip Specialist School</h5>]

In [22]:
house_df

Beds Baths Parking  Area                                      Address  \
0     2  None    None  None  605/455 Elizabeth Street Melbourne VIC 3000   
1  None  None       1  None     309/9 Commercial Road Melbourne VIC 3004   
2     2     2       1  None     314/499 St Kilda Road Melbourne VIC 3004   
3  None  None       1  None  Unit 1713/200 Spencer St Melbourne VIC 3000   
4  None  None    None  None         701/101 Therry St Melbourne VIC 3000   
5     2     2    None  None         816/101 Therry St Melbourne VIC 3000   
6     2     2       2  None     303/250 St Kilda Road Melbourne VIC 3004   
7  None  None       1  None        805/81 Queens Road Melbourne VIC 3004   
8     2     2       1  None   3902/220 Spencer Street Melbourne VIC 3000   
9  None  None    None  None   3405/288 Spencer Street Melbourne VIC 3000   

      Latitude    Longitude                                       Rent  \
0  -37.8084293  144.9602064                              $500 per week   
1  -37.8456125  144.9793561                              $500 per week   
2  -37.8431328  144.9793408                $900 per week (unfurnished)   
3   -37.816228  144.9532465                              $440 per week   
4  -37.8074149  144.9591227        NEW corner apartment Keep your bond   
5  -37.8074149  144.9591227  New apartment No bond needed Pets welcome   
6  -37.8259165  144.9705248                             $1395 Per Week   
7   -37.853804  144.9798785                                $380 weekly   
8  -37.8157157  144.9529559                              $650 per week   
9   -37.813775  144.9520948                                       $440   

                                              School  \
0  [Eltham College - Lonsdale Street Campus, Rive...   
1  [SEDA College (Victoria) - Tennis - Albert Par...   
2  [SEDA College (Victoria) - Tennis - Albert Par...   
3  [Eltham College - King Street Campus, Ozford C...   
4  [River Nile School, Simonds Catholic College, ...   
5  [River Nile School, Simonds Catholic College, ...   
6  [Hester Hornbrook Academy, Melbourne Grammar S...   
7  [Wesley College - St Kilda Road Junior Campus,...   
8  [Eltham College - King Street Campus, Ozford C...   
9  [Ozford College - Ozford College Campus, Ozfor...   

                                     School_distance          Location_1  \
0  [463.3611599786358, 588.5331765906494, 695.270...  [455 Elizabeth St]   
1  [196.76699981293854, 433.2955593802291, 726.22...   [9 Commercial Rd]   
2  [147.59638689110963, 586.8570067562501, 678.78...   [499 St Kilda Rd]   
3  [241.54098639384102, 397.47737499083695, 398.5...    [200 Spencer St]   
4  [444.35865271135503, 572.3778370522139, 577.38...         [Therry St]   
5  [444.35865271135503, 572.3778370522139, 577.38...         [Therry St]   
6  [745.7774466794033, 866.7220566327758, 961.355...                None   
7  [246.02949240830281, 586.9835254409847, 1034.4...      [81 Queens Rd]   
8  [291.8975714410087, 352.5634804374277, 353.655...    [220 Spencer St]   
9  [240.2562788817361, 241.10246190089674, 307.86...    [288 Spencer St]   

               Location_2 recent_rented_number recent_rented_price  \
0  [Melbourne, VIC, 3000]                   81                None   
1  [Melbourne, VIC, 3004]                   71                None   
2  [Melbourne, VIC, 3004]                  276                None   
3  [Melbourne, VIC, 3000]                  506                None   
4  [Melbourne, VIC, 3000]                 1380                None   
5  [Melbourne, VIC, 3000]                 1380                None   
6                    None                 None                None   
7  [Melbourne, VIC, 3004]                   78                None   
8  [Melbourne, VIC, 3000]                 1151                None   
9  [Melbourne, VIC, 3000]                  315                None   

  Converted_Rent  Type Postcode  
0            500   1.0     3000  
1            500   1.0     3004  
2            900   1.0  

In [23]:
# import required libraries
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, isnan, when, count, mean, udf, split, unix_timestamp, from_unixtime
from pyspark.sql.types import StringType, IntegerType

# init SparkSession class
spark = (
    # if available consider use yarn master node
    SparkSession.builder.master("local[*]") 
    
    # spark executor env configuration
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "16g")
    .config("spark.executor.cores", "2")
    .config("spark.executor.instances", "6")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    
    # jvm memory configuration
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "8g")
    
    # parquet file load configuration
    .config("spark.sql.repl.eagerEval.enabled", 'true')
    .config("spark.sql.parquet.cacheMetadata", 'true')
    
    # build the session
    .appName("Pyspark Start Template") # change app name here
    .getOrCreate()
)

# change default log level
spark.sparkContext.setLogLevel('ERROR')

# import libaries
import pandas as pd
import seaborn as sns

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/02 23:18:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Beds                  5 non-null      object 
 1   Baths                 4 non-null      object 
 2   Parking               6 non-null      object 
 3   Area                  0 non-null      object 
 4   Address               10 non-null     object 
 5   Latitude              10 non-null     object 
 6   Longitude             10 non-null     object 
 7   Rent                  10 non-null     object 
 8   School                10 non-null     object 
 9   School_distance       10 non-null     object 
 10  Location_1            9 non-null      object 
 11  Location_2            9 non-null      object 
 12  recent_rented_number  9 non-null      object 
 13  recent_rented_price   0 non-null      object 
 14  Converted_Rent        10 non-null     object 
 15  Type                  7 no

In [25]:
house_df.to_parquet('../data/1.parquet')

In [26]:
pdf = spark.read.parquet("../data/1.parquet", header=True)
pdf.limit(5)

+----+-----+-------+----+--------------------+-----------+-----------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------+----+--------+
|Beds|Baths|Parking|Area|             Address|   Latitude|  Longitude|                Rent|              School|     School_distance|        Location_1|          Location_2|recent_rented_number|recent_rented_price|Converted_Rent|Type|Postcode|
+----+-----+-------+----+--------------------+-----------+-----------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------+----+--------+
|   2| null|   null|null|605/455 Elizabeth...|-37.8084293|144.9602064|       $500 per week|[Eltham College -...|[463.361159978635...|[455 Elizabeth St]|[Melbourne, VIC, ...|                  81|               null|           500| 1.0|    3000|
|null| null|      1|null|309/9 Commercial ...|-37.8456125|144.9793561|       $500 per week|[SEDA College (Vi...|[196.766999812938...| [9 Commercial Rd]|[Melbourne, VIC, ...|                  71|               null|           500| 1.0|    3004|
|   2|    2|      1|null|314/499 St Kilda ...|-37.8431328|144.9793408|$900 per week (un...|[SEDA College (Vi...|[147.596386891109...| [499 St Kilda Rd]|[Melbourne, VIC, ...|                 276|               null|           900| 1.0|    3004|
|null| null|      1|null|Unit 1713/200 Spe...| -37.816228|144.9532465|       $440 per week|[Eltham College -...|[241.540986393841...|  [200 Spencer St]|[Melbourne, VIC, ...|                 506|               null|           440| 1.0|    3000|
|null| null|   null|null|701/101 Therry St...|-37.8074149|144.9591227|NEW corner apartm...|[River Nile Schoo...|[444.358652711355...|       [Therry St]|[Melbourne, VIC, ...|                1380|               null|              |null|    3000|
+----+-----+-------+----+--------------------+-----------+-----------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+-------------------+--------------+----+--------+

In [27]:
pdf.printSchema()

root
 |-- Beds: string (nullable = true)
 |-- Baths: string (nullable = true)
 |-- Parking: string (nullable = true)
 |-- Area: integer (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Rent: string (nullable = true)
 |-- School: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- School_distance: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Location_1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Location_2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- recent_rented_number: string (nullable = true)
 |-- recent_rented_price: integer (nullable = true)
 |-- Converted_Rent: string (nullable = true)
 |-- Type: double (nullable = true)
 |-- Postcode: string (nullable = true)



In [1]:
import os
os.chdir('../scripts')

from scrape_domain import *
from constants import *
from utils import *

In [2]:
vic_postcodes = constants.postcodes['VIC']

domain_links = set()

for i in tqdm(range(len(vic_postcodes))):
    current_links = domain_property_links(vic_postcodes[i])
    domain_links = domain_links | current_links
    
    print(len(current_links))

  0%|          | 1/1000 [00:13<3:46:46, 13.62s/it]

588


0it [00:00, ?it/s]
  0%|          | 2/1000 [00:13<1:35:11,  5.72s/it]

0


  0%|          | 3/1000 [00:15<1:03:30,  3.82s/it]

...

0


0it [00:00, ?it/s]
 34%|███▎      | 335/1000 [07:33<05:06,  2.17it/s]

0


 34%|███▎      | 335/1000 [07:36<15:05,  1.36s/it]


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
len(domain_links)

15354

In [ ]:
domain_properties_info(domain_links)

{'https://www.domain.com.au/6-9-eildon-road-st-kilda-vic-3182-16035949',
 'https://www.domain.com.au/14-kippax-court-mount-waverley-vic-3149-16032538',
 'https://www.domain.com.au/215-253-normanby-road-south-melbourne-vic-3205-16034152',
 'https://www.domain.com.au/14-newcombe-court-wantirna-south-vic-3152-16039701',
 'https://www.domain.com.au/99a-seventh-avenue-altona-north-vic-3025-16064939',
 'https://www.domain.com.au/1-3-everglade-crescent-roxburgh-park-vic-3064-16024258',
 'https://www.domain.com.au/154-morris-road-hoppers-crossing-vic-3029-16050164',
 'https://www.domain.com.au/4-137-coppin-street-richmond-vic-3121-16069075',
 'https://www.domain.com.au/57-grevillea-drive-mount-duneed-vic-3217-15015754',
 'https://www.domain.com.au/1-rossi-street-ivanhoe-vic-3079-16051631',
 'https://www.domain.com.au/2-20-surrey-road-south-yarra-vic-3141-16066354',
 'https://www.domain.com.au/2a-ann-court-aspendale-vic-3195-15997788',
 'https://www.domain.com.au/317-157-fitzroy-st-st-kilda-vic